In [1]:
import os
import glob
import sys

sys.path.insert(0, sys.path[0] + '/scripts')
from bert import *

In [2]:
mismatchable_params = set(['schedule_lin', 'schedule_cos', 'schedule_exp', 'schedule_none'])

def get_type(line):
    '''Determine the line type (function, class, or other/None)
        line: string of a line in python script
    '''
    if line[:3] == 'def':
        return 'def'
    if line[:5] == 'class':
        return 'class'
    return None

def script2doc(f_name):
    '''Parse single script to doc.
        f_name: file path for script
    '''
    def process_function():
        '''Parse function.'''
        nonlocal start, docs
        name = f[start].split('(', 1)[0].split()[-1]
        # while loop to find start of comment
        end = start
        while end < len(f):
            if f[end].strip().startswith("'''"):
                break
            end += 1
        # grab comment and params
        comment = f[end][:-1].strip().replace("'''", '')
        params = (''.join(f[start:end])).split('(', 1)[1].rsplit(')', 1)[0].replace(' ', '').replace('\n', '').split(',')
        if len(params) == 1 and not params[0]:
            params = []
            
        doc = list(map(lambda l: l.strip(), globals()[name].__doc__.split('\n')))
        comment = doc[0]
        com_params = {}
        if len(doc) > 1:
            for p in doc[1:-1]:
                k, v = p.split(':', 1)
                com_params[k] = v[1:] # whitespace after colon
        docs.append(('DEF', name, params, comment, com_params))
        
        assert comment[-1] == '.', print(name)
        if len(params) != len(com_params) and name not in mismatchable_params: 
            print(name, '\n', params, '\n', com_params, '\n')
        
    def process_class():
        '''Parse class.'''
        nonlocal start, docs
        name = f[start].split()[1].rsplit(')')[0] + ')'
        
        end, start = start+1, -1
        while end < len(f):
            if '__init__' in f[end] and start == -1:
                start = end
            if ')' in f[end]:
                break
            end += 1
        assert start != -1, name
        
        params = ''.join(f[start: end+1])
        params = params.split('(', 1)[1].rsplit(')', 1)[0]
        params = params.replace(' ', '').replace('\n', '')
        params = list(params)
        
        in_bracket, stored_commas = False, []
        for i, c in enumerate(params):
            if c == '(':
                in_bracket = True
            elif c == ')':
                in_bracket = False
            elif c == ',' and in_bracket:
                params[i] = '$' # temporarily replace , with $
        params = ''.join(params).split(',')[1:]
        for i in range(len(params)): 
            params[i] = params[i].replace('$', ',')
        start = end + 1
        
        doc = list(map(lambda l: l.strip(), globals()[name.split('(', 1)[0]].__init__.__doc__.split('\n')))
        comment = doc[0]
        com_params = {}
        if len(doc) > 1:
            for p in doc[1:-1]:
                k, v = p.split(':', 1)
                com_params[k] = v[1:]
        docs.append(('CLS', name, params, comment, com_params))
        
        assert comment[-1] == '.', print(name)
        if len(params) != len(com_params): print(name, '\n', params, '\n', com_params, '\n')
                
    docs = []
    with open(f_name, 'r') as f:
        f = list(f)
        start = 0
        while start < len(f):
            type_ = get_type(f[start])
            if type_ == 'def':
                process_function()
            elif type_ == 'class':
                process_class()
            start += 1
    return docs

In [3]:
def clean():
    '''Purge generated html directory.'''
    files = glob.glob('docs/assets/generated_htmls/*')
    for f_name in files:
        if f_name.endswith('txt'):
            os.remove(f_name)

def scripts2docs():
    '''Parse all scripts and return a mapping from scripts to documentations.'''
    all_docs = {}
#     all_docs['scripts/data_block.py'] = script2doc('scripts/data_block.py')
#     return all_docs
    for name in glob.glob('scripts/*'):
        if name.endswith('py'):
            all_docs[name] = script2doc(name)
    return all_docs

In [4]:
template1 = '''
                                <div class="section-block-small">
                                    <div class="doc-block">
                                        <table>
                                           <td><span class="label">&nbsp;{}</span></td>
                                            <td><span class="name">{}</span></td>
                                            <td class="expand"><span class="params">({})</span></td>
                                        </table>
                                        <p><span class="desc">{}</span></p>
                                    </div>
                                </div>'''


template2_head = '''                                <div class="section-block-small">
                                    <div class="doc-block">
                                        <table>
                                           <td><span class="label">&nbsp;{}</span></td>
                                            <td><span class="name">{}</span></td>
                                            <td class="expand"><span class="params">({})</span></td>
                                        </table>
                                        <p><span class="desc">{}</span></p>
                                        <ul class="list">'''

template2_tail = '''
                                        </ul>
                                    </div>
                                </div>'''

In [5]:
def doc2html(doc):
    '''Convert single doc to html.
        doc: documentation pulled from python script
    '''
    html = []
    for type_, name, params, comment, com_params in doc:
        if not com_params:
            html.append(template1.format(type_, name, ', '.join(params), comment))
        else:
            cur = template2_head.format(type_, name, ', '.join(params), comment)
            for k, v in com_params.items():
                cur += f"\n                                           <li><b>{k}</b>: {v}</li>"
            cur += template2_tail
            html.append(cur)
    return '\n'.join(html)

def docs2htmls(all_docs):
    '''Convert all docs to a map from file name to html files.
        all_docs: a map of script names to documentations
    '''
    htmls = {}
    for name, doc in all_docs.items():
        htmls[name] = doc2html(doc)
    return htmls

def store_htmls(htmls):
    '''Store htmls to docs/assets/generated_htmls.
        htmls: a map of script names to html files
    '''
    for name, html in htmls.items():
        name = 'docs/assets/generated_htmls/' + name.split('/')[1][:-3] + '.html'
        with open(name, 'w') as f:
            f.write(html)

In [6]:
def scripts2htmls():
    '''Wrapper function for going from python scripts to html'''
    all_docs = scripts2docs()
    htmls = docs2htmls(all_docs)
    store_htmls(htmls)
    return htmls

In [7]:
clean()
htmls = scripts2htmls()